In [1]:
import pandas
from RexR import *
import _helpers
Rocket = RexR()

Firing up RexR!


In [ ]:
##### LOAD IN DATA
###################

In [2]:
Rocket.load_probeset_data();

In [3]:
class_model = Rocket.classify_treatment_model(model='SVM', 
                                              features = 'all', 
                                              grouping= 'first', 
                                              n_splits = 10, 
                                              reduction = None)

++++++++++++++++++++++++++++++  RESULTS FOR CLASSIFICATION WITH GENOMIC DATA ++++++++++++++++++++++++++++++
0.857142857143
0.714285714286
0.714285714286
0.5
0.5
1.0
0.6
0.4
0.8
0.8
++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++  RESULTS FOR CLASSIFICATION INCLUDING PATIENT DATA ++++++++++++++++++++++++++++++


IndexError: too many indices for array

In [ ]:
test = np.append([[4, 5, 6], [7, 8, 9]])